# Script mode on Amazon SageMaker


Sript mode is a way to work on Machine learning on Amazon Sagemaker only providing the script for processing, training or inference. In this notebook we will focuse on the lowest level of the Script mode usage that is to say using the base class provided by Amzon SageMaker.

This notebook will follow each parts of a usual ML workflow with some explaination of the different SageMaker command used.

First we want to import the different packages and load the data to S3 if it is not already done.

In [ ]:
import boto3
import sagemaker

import pandas as pd
import numpy as np
import os

In [ ]:
#Manage interactions with the Amazon SageMaker APIs and any other AWS services needed
session = sagemaker.Session()
#see the region in which we work
region = session.boto_region_name
print("AWS Region : {}".format(region))
#get the role of the running session
role = sagemaker.get_execution_role()
#get the bucket name of the session
bucket = session.default_bucket()

We can now push the data to S3 :

In [ ]:
#Upload the dataset to S3
prefix = "data_script_mode"
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'data/dataset.csv')).upload_file('predictive_maintenance.csv')

## Processing

This graph sum-up how SageMaker handles the processing task :

<img src="images/smprocess.PNG" width="600" height="400">

A processing job requires the specification of a path to an input S3 bucket that holds the data to be processed. The job utilizes a provided script to perform the processing task. The resulting output data is then stored in a separate S3 path.

S3 effectively manages the job environment by utilizing Docker containers. These containers can either be pre-built containers provided by SageMaker, which are accessible on the Elastic Container Registry (ECR), or custom containers created from custom images that must be pushed to ECR.





SageMaker provide different class to instantiate some processing object to run processing job : 

<img src="images/processing.PNG" width="700" height="500">

We will use the Processor class. To do so we first need to have a docker image in which we get the sript we want to run for processing. Let's build such an image and push it to ECR :

In [ ]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [ ]:
./build_and_push.sh

One we have our image pushed on ECR, we need to implement a Processor object which will be used to launch the processing job, for more information about the processing class see https://sagemaker.readthedocs.io/en/stable/api/training/processing.html

The ProcessingInput class represents an input source for a processing job in Amazon SageMaker. It encapsulates information about the input data location, such as the S3 bucket path, and any optional configurations or preprocessing steps required before the processing job begins.
The ProcessingOutput class represents an output destination for a processing job. It contains information about where the processed data should be stored, including the S3 bucket path and any optional configurations or post-processing steps.
We can add some argument which are passed with argparse to our processing script. See the processing.py file to have more information about the architecture of the code.

In [ ]:
from sagemaker.processing import Processor
from sagemaker.processing import ProcessingInput, ProcessingOutput

#first we instanciate the processor with the image uri of our ECR image, and as described above, we need to provide the entrypoint of the docker container
processor = Processor(
    role = role,
    image_uri = "222978838857.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-container",
    instance_count = 1,
    instance_type = "local",
    entrypoint = ["python3", "processing.py"]
    )
#The path of our S3 bucket
bucket_path = 's3://{}'.format(bucket)

#we then launch the processing job
processor.run(
    inputs=[ProcessingInput(source=f"{bucket_path}/{prefix}/data/dataset.csv", destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test"),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)

One the job is completed, we can retrieve some information about it, espacially get the S3 path of the output data so that we can use it for the training :

In [ ]:
preprocessing_job_description = processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]

for output in output_config["Outputs"]:
    if output["OutputName"] == "train_data":
        preprocessed_training_data = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "test_data":
        preprocessed_test_data = output["S3Output"]["S3Uri"]
        
#Observe the processed data 
training_features = pd.read_csv(preprocessed_training_data + "/dataset_train.csv", index_col = "UDI",nrows=10)
print("Training features shape: {}".format(training_features.shape))
training_features.drop(["Target"], axis=1)

## Training

The training part is similar to the processing part in the code but has some difference on the way SageMaker handles the task.

<img src="images/training.PNG" width="500" height="700">

(1) On the Jupyter Notebook, you need to instanciate the training object to make the API call to SageMaker, push the data to S3 if needed, and push the image to ECR if needed.


(2) One you run the fit method of the estimator you instanciated, you call the SageMaker API with the create_training_job request (see : https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_training_job.html), SageMaker launch the EC2 instance with the information you provided in the request (which is basicaly a json file since we work with RESTful API)

(3) The training job run on the EC2 instance, when it has finished, it stores the output on a S3 bucket (model artifact, logs...) and shutdown every instance.

(4) All the training outputs are available on a S3 bucket and the model is ready to be deployed on an endpoint



In our case, we load a SageMaker image from ECR and use it for training. Then we just have to provide the training script as the entrypoit.

In [ ]:
from sagemaker import image_uris
from sagemaker.estimator import Estimator

training_image = image_uris.retrieve(framework='sklearn',region='us-east-1',version='1.2-1',image_scope='training')


metric = {
    'Name' : 'Accuracy', 'Regex' : 'Accuracy : ([0-9\\.]+)'
}

estimator = Estimator(
    role = role,
    instance_count = 1,
    instance_type = "local",
    base_job_name = "job",
    image_uri = training_image,
    entry_point = "train.py",
    metric_definitions = [metric]
)

estimator.set_hyperparameters(
    C = 1,
    kernel = "poly"
)


estimator.fit({"train" : preprocessed_training_data, "test" : preprocessed_test_data})

As for the processing, we can retrieve some information about the job. For example, the S3 path of our model to use it for inference if we want to use a custom inference script.

In [ ]:
training_job_description = estimator.jobs[-1].describe()
training_job_description
model_data_s3_uri = "{}".format(training_job_description["ModelArtifacts"]["S3ModelArtifacts"])
model_data_s3_uri

## Deploy

Once the training is done, our model is ready to be deployed to and enpoint. We could directly use the deploy() method on our estimator but here we have not implemented an inference part in our training script and we want to use a different image for training and inference.
We will use the class Model to deploy our model to an enpoint :

In [ ]:
from sagemaker.model import Model

inference_image = image_uris.retrieve(framework='sklearn',region='us-east-1',version='1.2-1',image_scope='inference')

model = Model(
    image_uri = inference_image,
    model_data = model_data_s3_uri,
    role = role,
    entry_point = "inference.py",
)


In [ ]:

predictor = model.deploy(    
    initial_instance_count = 1,
    instance_type = "local",
    endpoint_name = "myendpoint")

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()